In [3]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m1.pt')

data.x = torch.randn((900, 100))
train_data, val_data, test_data = RandomLinkSplit(num_val=0.0,
                                                  num_test=0.1,
                                                  is_undirected=True)(data)

train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
    num_workers=6, persistent_workers=True,
)
model = GraphSAGE(
    train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        batch.edge_index = batch.edge_index % 900
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


times = []
for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 37.4164,AUC: 0.6925,AP: 0.6403
Epoch: 002, Loss: 29.4341,AUC: 0.7454,AP: 0.7017
Epoch: 003, Loss: 27.5449,AUC: 0.7696,AP: 0.7369
Epoch: 004, Loss: 26.6442,AUC: 0.7921,AP: 0.7667
Epoch: 005, Loss: 26.1253,AUC: 0.8082,AP: 0.7890
Epoch: 006, Loss: 25.5689,AUC: 0.8237,AP: 0.8091
Epoch: 007, Loss: 25.1949,AUC: 0.8262,AP: 0.8146
Epoch: 008, Loss: 24.9006,AUC: 0.8368,AP: 0.8293
Epoch: 009, Loss: 24.5322,AUC: 0.8409,AP: 0.8358
Epoch: 010, Loss: 24.6486,AUC: 0.8446,AP: 0.8396
Epoch: 011, Loss: 24.4770,AUC: 0.8528,AP: 0.8480
Epoch: 012, Loss: 24.2695,AUC: 0.8490,AP: 0.8448
Epoch: 013, Loss: 24.1306,AUC: 0.8573,AP: 0.8523
Epoch: 014, Loss: 24.0202,AUC: 0.8610,AP: 0.8558
Epoch: 015, Loss: 23.9486,AUC: 0.8631,AP: 0.8588
Epoch: 016, Loss: 23.6919,AUC: 0.8649,AP: 0.8616
Epoch: 017, Loss: 23.6120,AUC: 0.8683,AP: 0.8668
Epoch: 018, Loss: 23.7323,AUC: 0.8673,AP: 0.8674
Epoch: 019, Loss: 23.5517,AUC: 0.8778,AP: 0.8774
Epoch: 020, Loss: 23.4587,AUC: 0.8764,AP: 0.8771
Epoch: 021, Loss: 23

In [4]:
z1 = model(train_data.x, train_data.edge_index)
z1.shape

torch.Size([900, 128])

In [5]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m2.pt')

data.x = torch.randn((900, 100))
train_data, val_data, test_data = RandomLinkSplit(num_val=0.0,
                                                  num_test=0.1,
                                                  is_undirected=True)(data)

train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
    num_workers=6, persistent_workers=True,
)
model = GraphSAGE(
    train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        batch.edge_index = batch.edge_index % 900
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


times = []
for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 10.2854,AUC: 0.8103,AP: 0.7534
Epoch: 002, Loss: 6.4928,AUC: 0.8387,AP: 0.8034
Epoch: 003, Loss: 5.8345,AUC: 0.8695,AP: 0.8503
Epoch: 004, Loss: 5.5820,AUC: 0.8820,AP: 0.8732
Epoch: 005, Loss: 5.3051,AUC: 0.8962,AP: 0.8912
Epoch: 006, Loss: 5.2046,AUC: 0.9003,AP: 0.8992
Epoch: 007, Loss: 5.1720,AUC: 0.9034,AP: 0.9076
Epoch: 008, Loss: 4.9935,AUC: 0.9130,AP: 0.9182
Epoch: 009, Loss: 4.9476,AUC: 0.9120,AP: 0.9179
Epoch: 010, Loss: 4.8999,AUC: 0.9171,AP: 0.9268
Epoch: 011, Loss: 4.7139,AUC: 0.9159,AP: 0.9273
Epoch: 012, Loss: 4.7490,AUC: 0.9209,AP: 0.9312
Epoch: 013, Loss: 4.7165,AUC: 0.9190,AP: 0.9310
Epoch: 014, Loss: 4.6491,AUC: 0.9189,AP: 0.9331
Epoch: 015, Loss: 4.5922,AUC: 0.9207,AP: 0.9341
Epoch: 016, Loss: 4.5545,AUC: 0.9239,AP: 0.9370
Epoch: 017, Loss: 4.5729,AUC: 0.9195,AP: 0.9350
Epoch: 018, Loss: 4.5448,AUC: 0.9218,AP: 0.9379
Epoch: 019, Loss: 4.5249,AUC: 0.9230,AP: 0.9399
Epoch: 020, Loss: 4.5216,AUC: 0.9212,AP: 0.9402
Epoch: 021, Loss: 4.5420,AUC: 0.9218,AP

In [6]:
z2 = model(train_data.x, train_data.edge_index)
z2.shape

torch.Size([900, 128])

In [7]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m3.pt')

data.x = torch.randn((900, 100))
train_data, val_data, test_data = RandomLinkSplit(num_val=0.0,
                                                  num_test=0.1,
                                                  is_undirected=True)(data)

train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
    num_workers=6, persistent_workers=True,
)
model = GraphSAGE(
    train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        batch.edge_index = batch.edge_index % 900
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


times = []
for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 7.4458,AUC: 0.8248,AP: 0.7799
Epoch: 002, Loss: 4.2932,AUC: 0.8691,AP: 0.8472
Epoch: 003, Loss: 3.9417,AUC: 0.8697,AP: 0.8637
Epoch: 004, Loss: 3.5727,AUC: 0.8925,AP: 0.8926
Epoch: 005, Loss: 3.3667,AUC: 0.8917,AP: 0.8982
Epoch: 006, Loss: 3.2432,AUC: 0.9049,AP: 0.9146
Epoch: 007, Loss: 3.3137,AUC: 0.8995,AP: 0.9131
Epoch: 008, Loss: 3.1570,AUC: 0.9072,AP: 0.9209
Epoch: 009, Loss: 3.1824,AUC: 0.9062,AP: 0.9210
Epoch: 010, Loss: 3.1550,AUC: 0.9145,AP: 0.9291
Epoch: 011, Loss: 3.1284,AUC: 0.9058,AP: 0.9232
Epoch: 012, Loss: 3.1092,AUC: 0.9095,AP: 0.9272
Epoch: 013, Loss: 3.0678,AUC: 0.9063,AP: 0.9261
Epoch: 014, Loss: 3.0919,AUC: 0.9110,AP: 0.9308
Epoch: 015, Loss: 2.9214,AUC: 0.9118,AP: 0.9318
Epoch: 016, Loss: 3.0100,AUC: 0.9136,AP: 0.9336
Epoch: 017, Loss: 2.9838,AUC: 0.9118,AP: 0.9333
Epoch: 018, Loss: 2.9792,AUC: 0.9137,AP: 0.9355
Epoch: 019, Loss: 3.0149,AUC: 0.9077,AP: 0.9319
Epoch: 020, Loss: 2.9759,AUC: 0.9129,AP: 0.9364
Epoch: 021, Loss: 2.9680,AUC: 0.9089,AP:

In [8]:
z3 = model(train_data.x, train_data.edge_index)
z3.shape

torch.Size([900, 128])

In [9]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m4.pt')

data.x = torch.randn((900, 100))
train_data, val_data, test_data = RandomLinkSplit(num_val=0.0,
                                                  num_test=0.1,
                                                  is_undirected=True)(data)

train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
    num_workers=6, persistent_workers=True,
)
model = GraphSAGE(
    train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        batch.edge_index = batch.edge_index % 900
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


times = []
for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 6.8096,AUC: 0.8941,AP: 0.8626
Epoch: 002, Loss: 3.7950,AUC: 0.9230,AP: 0.9242
Epoch: 003, Loss: 3.3581,AUC: 0.9251,AP: 0.9348
Epoch: 004, Loss: 3.1450,AUC: 0.9336,AP: 0.9481
Epoch: 005, Loss: 3.1139,AUC: 0.9338,AP: 0.9504
Epoch: 006, Loss: 2.9684,AUC: 0.9384,AP: 0.9561
Epoch: 007, Loss: 2.9562,AUC: 0.9362,AP: 0.9548
Epoch: 008, Loss: 2.9521,AUC: 0.9394,AP: 0.9581
Epoch: 009, Loss: 2.9351,AUC: 0.9388,AP: 0.9581
Epoch: 010, Loss: 2.8882,AUC: 0.9410,AP: 0.9604
Epoch: 011, Loss: 2.8039,AUC: 0.9407,AP: 0.9601
Epoch: 012, Loss: 2.8790,AUC: 0.9397,AP: 0.9590
Epoch: 013, Loss: 2.7647,AUC: 0.9397,AP: 0.9595
Epoch: 014, Loss: 2.8617,AUC: 0.9398,AP: 0.9602
Epoch: 015, Loss: 2.7679,AUC: 0.9389,AP: 0.9598
Epoch: 016, Loss: 2.7982,AUC: 0.9393,AP: 0.9597
Epoch: 017, Loss: 2.7176,AUC: 0.9404,AP: 0.9607
Epoch: 018, Loss: 2.8252,AUC: 0.9377,AP: 0.9594
Epoch: 019, Loss: 2.7471,AUC: 0.9400,AP: 0.9608
Epoch: 020, Loss: 2.7483,AUC: 0.9405,AP: 0.9610
Epoch: 021, Loss: 2.7538,AUC: 0.9414,AP:

In [10]:
z4 = model(train_data.x, train_data.edge_index)
z4.shape

torch.Size([900, 128])

In [11]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m5.pt')

data.x = torch.randn((900, 100))
train_data, val_data, test_data = RandomLinkSplit(num_val=0.0,
                                                  num_test=0.1,
                                                  is_undirected=True)(data)

train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
    num_workers=6, persistent_workers=True,
)
model = GraphSAGE(
    train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        batch.edge_index = batch.edge_index % 900
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


times = []
for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 5.1830,AUC: 0.7893,AP: 0.7621
Epoch: 002, Loss: 2.8774,AUC: 0.8879,AP: 0.8671
Epoch: 003, Loss: 2.3904,AUC: 0.8999,AP: 0.8858
Epoch: 004, Loss: 2.1730,AUC: 0.9034,AP: 0.8996
Epoch: 005, Loss: 2.0577,AUC: 0.9072,AP: 0.9075
Epoch: 006, Loss: 2.0629,AUC: 0.9102,AP: 0.9114
Epoch: 007, Loss: 2.0658,AUC: 0.9063,AP: 0.9102
Epoch: 008, Loss: 1.9709,AUC: 0.9090,AP: 0.9107
Epoch: 009, Loss: 1.9070,AUC: 0.9144,AP: 0.9140
Epoch: 010, Loss: 1.9540,AUC: 0.9129,AP: 0.9137
Epoch: 011, Loss: 1.8977,AUC: 0.9105,AP: 0.9184
Epoch: 012, Loss: 1.8991,AUC: 0.9135,AP: 0.9279
Epoch: 013, Loss: 1.8641,AUC: 0.9116,AP: 0.9265
Epoch: 014, Loss: 1.7728,AUC: 0.9125,AP: 0.9288
Epoch: 015, Loss: 1.8545,AUC: 0.9124,AP: 0.9304
Epoch: 016, Loss: 1.8082,AUC: 0.9075,AP: 0.9277
Epoch: 017, Loss: 1.8366,AUC: 0.9084,AP: 0.9286
Epoch: 018, Loss: 1.8287,AUC: 0.9080,AP: 0.9288
Epoch: 019, Loss: 1.8158,AUC: 0.9059,AP: 0.9293
Epoch: 020, Loss: 1.8177,AUC: 0.9066,AP: 0.9312
Epoch: 021, Loss: 1.7836,AUC: 0.9037,AP:

In [12]:
z5 = model(train_data.x, train_data.edge_index)
z5.shape

torch.Size([900, 128])

In [13]:
z_1_5 = torch.cat([z1, z2, z3, z4, z5], 1)
z_1_5.shape

torch.Size([900, 640])

In [14]:
# torch.save(z_1_5, '/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/z_1_5_gs.pt')

In [17]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
# device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m6.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.0,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

print(train_data.edge_index.shape)


@torch.no_grad()
def test():
    data = train_data
    z = z_1_5  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


AUC, AP = test()
print(f'AUC: {AUC:.4f},AP: {AP:.4f}')

torch.Size([2, 584])
AUC: 0.9541,AP: 0.9711
